## Json dump

In [11]:
from docling.document_converter import DocumentConverter
import json

source = r"/home/ubuntu/Desktop/OmniPDF/sample-files/o_level_paper_2.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
data = result.document.export_to_dict()

with open('./result.json', 'w') as fp:
    json.dump(data, fp, indent=4)

/home/ubuntu/Desktop/OmniPDF/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
from docling.document_converter import DocumentConverter
import json

source = r"/home/ubuntu/Desktop/OmniPDF/sample-files/Email to Area Lead for Approval_Extended.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
data = result.document.export_to_dict()

with open('./result.json', 'w') as fp:
    json.dump(data, fp, indent=4)

## Concat

In [12]:
for ref in ['body', 'groups']:
    if ref in data.keys():
        data.pop(ref)
with open('./result.json', 'w') as fp:
    json.dump(data, fp, indent=4)

## Translate

In [7]:
import requests

system_prompt = "Translate the following from english to chinese"

In [19]:
LLM_URL = "http://192.168.1.108:80/v1/chat/completions" #chat/completions

TOKEN = "token-abc123"

def translate(prompt):
    r = requests.post(
        LLM_URL,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {TOKEN}"
        },
        json={
            "model": "qwen2.5",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0
        }
    )

    # print(r.json()["choices"][0]["message"]["content"])
    return r.json()["choices"][0]["message"]["content"]

In [20]:
translate('hello')

'你好'

In [22]:
i = 0
for para in data['texts']:
    print(f"Paragraph {i}")
    print(para["text"])
    print(translate(para["text"]))
    para["translation"] = translate(para["text"])
    i += 1

with open('./translated.json', 'w') as fp:
    json.dump(data, fp, indent=4)


Paragraph 0
Cambridge O Level
剑桥O水准考试
Paragraph 1
*0123456789*
*0123456789* 

这个字符串在英语和中文中是一样的，因为它只包含数字和星号。
Paragraph 2
ENGLISH LANGUAGE
英语语言
Paragraph 3
1123/01
1123/01 可以直译为中文的“1123/01”，因为这看起来像一个日期或者编号，通常不需要翻译。如果您有特定的上下文或需要进一步的帮助，请提供更多信息。
Paragraph 4
Paper 1 Reading
第一部分：阅读理解
Paragraph 5
For examination from 2024
从2024年开始的考试
Paragraph 6
SPECIMEN INSERT
样品插入物
Paragraph 7
2 hours
2小时
Paragraph 8
INFORMATION
信息
Paragraph 9
● This insert contains the reading passages.
● 此插页包含阅读篇章。
Paragraph 10
● You may annotate this insert and use the blank spaces for planning. Do not write your answers on the insert.
●您可以在本插入页上做标注，并使用空白处进行规划。但请不要在插入页上写下您的答案。
Paragraph 11
*0123456789*
*0123456789* 

这个字符串在英语和中文中是一样的，因为它只包含数字和星号。
Paragraph 12
This document has 6 pages. Any blank pages are indicated.
该文件共有6页。空白页已在文档中注明。
Paragraph 13
© Cambridge University Press & Assessment 2021
© 版权 Cambridge University Press & Assessment 2021
Paragraph 14
[Turn over
[翻到下一页]
Paragraph 15
2
2翻译成中文还是2，因为这是一个数字，中英文表达一致。如果你有

## PDF Generation

In [10]:
import pymupdf

doc = pymupdf.open("/home/ubuntu/Desktop/OmniPDF/sample-files/o_level_paper_2.pdf")
for page in doc:
    print(page.number)

0
1
2
3
4
5


In [ ]:
import pymupdf

doc = pymupdf.open("/home/ubuntu/Desktop/OmniPDF/sample-files/o_level_paper_2.pdf")
for page in doc:
    # print(f"{page.rect}\n") 
    blocks = page.get_text("blocks")  # list of tuples: (x0, y0, x1, y1, "text", block_no, block_type)
    for block in blocks:
        rect = block[:4]  # (x0, y0, x1, y1)
        text = block[4]

        # page.draw_rect(rect, color=(1, 1, 1), fill=(1, 1, 1)) # color=(1, 1, 1)
        page.draw_rect(rect, fill=(1, 1, 1))

        print(f"Rect: {rect} | Text: {text}")

doc.save("./output_white_boxed.pdf")

In [65]:
def filter_texts_by_page(data, total_pages):
    output = {}
    for page_num in range(1, total_pages + 1):
        page_lst = []
        for text in data["texts"]:
            if any(p["page_no"] == page_num for p in text.get("prov", [])):
                page_lst.append(text)
        output[page_num] = (page_lst)
    return output

# Example usage:
total_pages = len(data["pages"])
print(total_pages)

filtered_texts = filter_texts_by_page(data, total_pages)

# for t in filtered_texts:
#     print(t["text"], t["prov"][0]["bbox"])

with open('./test.json', 'w') as fp:
    json.dump(filtered_texts, fp, indent=4)

6


In [74]:
import pymupdf

doc = pymupdf.open("/home/ubuntu/Desktop/OmniPDF/sample-files/o_level_paper_2.pdf")
for page in doc:
    print(f"{page.number}\n") 
    blocks = page.get_text("blocks")  # list of tuples: (x0, y0, x1, y1, "text", block_no, block_type)
    for block in blocks:
        rect = block[:4]  # (x0, y0, x1, y1)
        text = block[4]

        page.add_redact_annot(rect)

    print(page.rect[3])
    page.apply_redactions()
    page.clean_contents()
    try:
        page_lst = filtered_texts[page.number + 1]
        for i in page_lst:
            new_text = i["text"]
            bbox = i["prov"][0]['bbox']
            coords = (bbox["l"], (page.rect[3]- bbox["t"]), bbox["r"], page.rect[3]- bbox["b"])

            print(f"Text: {new_text}")
            print(f"Bbox: {coords}")

            page.draw_rect(coords, color=(1, 0, 1))
            status = page.insert_htmlbox(coords, new_text) # fontsize=font_size, fontname=font_name, color=text_color, align=0
            print(f"Status: {status}")
    except KeyError:
        pass
doc.save("./output_white_boxed.pdf", garbage=3, deflate=True)

0

842.0
Text: Cambridge O Level
Bbox: (56.7, 103.00300000000004, 220.747, 123.03099999999995)


TypeError: insert_htmlbox() got an unexpected keyword argument 'color'